In [128]:
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


movie_data_path = Path("source_data/movie_data.csv")
movie_data_df = pd.read_csv(movie_data_path)
movie_data_df.head()


/var/folders/v4/v1wh2j7n7_v1r1l0xj84m9xw0000gp/T/ipykernel_93301/1429549622.py:8: DtypeWarning: Columns (0,1,7,8,15,105,112,113,114,115,116,117,118,119,120,150,153,154,155,156,157,158,167,168,169,170,173,174,175,177,178,179,181,182,183,185,186,187,188,189) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_data_df = pd.read_csv(movie_data_path)


array(['Science Fiction', 'Horror', 'Drama', 'Thriller', 'Comedy',
       'Family', 'Fantasy', 'Action', 'Romance', 'Adventure', 'Mystery',
       'Crime', 'History', nan, 'Animation', 'TV Movie', 'Music', 'War',
       'Western', 'Documentary', '13', '15'], dtype=object)

In [122]:
# movie_data_df = movie_data_df[['budget', 'revenue', 'runtime', 'popularity', 'vote_average', 'vote_count', 'release_date', 'title', 'id', 'genres_0_name', 'genres_1_name', 'genres_2_name', 'production_companies_0_name', 'production_companies_1_name', 'production_companies_2_name', 'original_language', 'production_countries_0_name']]

movie_data_df = movie_data_df[['budget', 'revenue', 'runtime', 'popularity', 'vote_average', 'genres_0_name', 'genres_1_name', 'genres_2_name']]

ohe = OneHotEncoder()

categorical_cols = ['genres_0_name', 'genres_1_name', 'genres_2_name']
encoded_genres = ohe.fit_transform(movie_data_df[categorical_cols]).toarray()
encoded_genres_df = pd.DataFrame(encoded_genres, columns=ohe.get_feature_names_out(categorical_cols))

movie_data_df = pd.concat([movie_data_df.drop(columns=categorical_cols), encoded_genres_df], axis=1)
movie_data_df = movie_data_df.drop(columns=['genres_2_name_nan', 'genres_0_name_2016-09-01', 'genres_0_name_2024-10-26'])

movie_data_df.head()

movie_data_df = movie_data_df.dropna()



In [123]:
movie_data_df['profit'] = pd.to_numeric(movie_data_df['revenue']) - pd.to_numeric(movie_data_df['budget'])
movie_data_df = movie_data_df[pd.to_numeric(movie_data_df['budget']) > 0]
movie_data_df['roi'] = pd.to_numeric(movie_data_df['profit']) / pd.to_numeric(movie_data_df['budget'])
movie_data_df.sort_values(by=['profit', 'vote_average', 'popularity'], ascending=False, inplace=True)

vote_average_theshold = movie_data_df['vote_average'].quantile(0.75)
popularity_theshold = movie_data_df['popularity'].quantile(0.75)

movie_data_df.head()


,budget,revenue,runtime,popularity,vote_average,genres_0_name_Action,genres_0_name_Adventure,genres_0_name_Animation,genres_0_name_Comedy,genres_0_name_Crime,...,genres_2_name_Music,genres_2_name_Mystery,genres_2_name_Romance,genres_2_name_Science Fiction,genres_2_name_TV Movie,genres_2_name_Thriller,genres_2_name_War,genres_2_name_Western,profit,roi
322,237000000,2.923706e+09,162.0,129.244,7.583,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.686706e+09,11.336312
275,356000000,2.799439e+09,181.0,141.577,8.250,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.443439e+09,6.863593
199,200000000,2.264162e+09,194.0,166.361,7.906,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.064162e+09,10.320812
250,460000000,2.320250e+09,192.0,148.255,7.620,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.860250e+09,4.044022
1205,245000000,2.068224e+09,136.0,64.763,7.300,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.823224e+09,7.441729


Classification - Define success
Turn target into binary - use model to figure it out


In [126]:
movie_data_train, movie_data_test = train_test_split(movie_data_df, test_size=0.2, random_state=42)
movie_data_train.to_csv("movie_data_train.csv", index=False)
movie_data_test.to_csv("movie_data_test.csv", index=False)
